In [ ]:
import cv2
import numpy as np
import math

def count_fingers(contour, drawing):
    hull = cv2.convexHull(contour, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(contour, hull)
        if defects is None:
            return 0
        
        finger_count = 0
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            # Calculate the angle
            a = math.dist(start, end)
            b = math.dist(start, far)
            c = math.dist(end, far)
            angle = math.acos((b**2 + c**2 - a**2) / (2*b*c)) * (180/np.pi)

            if angle <= 90:
                finger_count += 1
                cv2.circle(drawing, far, 8, [211, 84, 0], -1)
        return finger_count
    return 0

def classify_gesture(finger_count):
    if finger_count == 0:
        return "rock"
    elif finger_count == 2:
        return "scissors"
    elif finger_count >= 4:
        return "paper"
    else:
        return "unknown"

if __name__ == "__main__":
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        drawing = frame.copy()

        # Convert to HSV and apply skin color range
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)
        mask = cv2.inRange(hsv, lower_skin, upper_skin)

        # Blur and threshold
        mask = cv2.GaussianBlur(mask, (5,5), 0)
        mask = cv2.dilate(mask, None, iterations=2)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        gesture = "No hand"

        if contours:
            max_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(max_contour) > 1000:
                hull = cv2.convexHull(max_contour)
                cv2.drawContours(drawing, [hull], -1, (0, 255, 0), 2)

                finger_count = count_fingers(max_contour, drawing)
                gesture = classify_gesture(finger_count)

        # Display Gesture
        cv2.putText(drawing, f"Gesture: {gesture}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 2)

        # Display Exit Instruction
        cv2.putText(drawing, "Press 'q' to Exit", (10, 450), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255, 255, 255), 2)

        cv2.imshow('RPS without MediaPipe', drawing)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting...")
            break

    cap.release()
    cv2.destroyAllWindows()
